In [1]:
from line_solver import *
import numpy as np
import time
import os
GlobalConstants.set_verbose(VerboseLevel.SILENT)

# This example is temporarily disabled (matching MATLAB comment)
# Clear variables (equivalent to MATLAB's clear solver AvgTable)
solver = None
AvgTable = None

In [ ]:
print('This example illustrates the solution of a moderately large LQN.')

# MATLAB: cwd = fileparts(which(mfilename));
# MATLAB: model = LayeredNetwork.parse_xml([cwd,filesep,'lqn_ofbiz.xml']);
cwd = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else os.getcwd()
xml_path = os.path.join(cwd, 'lqn_ofbiz.xml')
#xml_path = os.path.join(cwd, 'lqn_ofbiz_fcfs.xml')

try:
    # Try to parse XML file like MATLAB
    model = LayeredNetwork()
    model.parse_xml(xml_path)
    print(f'Successfully loaded model from {xml_path}')
except Exception as e:
    print(f'XML parsing failed: {e}')
    print('Creating representative large layered model programmatically')
    
    # Create a representative large layered model (ofbizExample substitute)
    model = LayeredNetwork('LargeLayeredModel')
    
    # Create multiple processors (representative of ofbiz complexity)
    processors = {}
    for i in range(1, 6):
        processors[i] = Processor(model, f'P{i}', i, SchedStrategy.PS)
    
    # Create multiple tasks (representative structure)
    tasks = {}
    entries = {}
    
    # Client layer
    tasks[1] = Task(model, 'ClientTask', 10, SchedStrategy.REF).on(processors[1]).set_think_time(Exp.fit_mean(1.0))
    entries[1] = Entry(model, 'ClientEntry').on(tasks[1])
    
    # Service layers
    for i in range(2, 5):
        tasks[i] = Task(model, f'ServiceTask{i}', 1, SchedStrategy.FCFS).on(processors[i]).set_think_time(Immediate())
        entries[i] = Entry(model, f'ServiceEntry{i}').on(tasks[i])
    
    # Database layer
    tasks[5] = Task(model, 'DatabaseTask', 1, SchedStrategy.FCFS).on(processors[5]).set_think_time(Immediate())
    entries[5] = Entry(model, 'DatabaseEntry').on(tasks[5])
    
    # Define activities and call structure (representative of ofbiz complexity)
    activities = {}
    
    # Client activities
    activities[1] = Activity(model, 'ClientActivity', Exp.fit_mean(0.1)).on(tasks[1]).bound_to(entries[1])
    activities[2] = Activity(model, 'ClientCall1', Immediate()).on(tasks[1]).synch_call(entries[2], 1)
    activities[3] = Activity(model, 'ClientCall2', Immediate()).on(tasks[1]).synch_call(entries[3], 1)
    activities[4] = Activity(model, 'ClientCall3', Immediate()).on(tasks[1]).synch_call(entries[4], 1)
    
    # Service activities
    for i in range(2, 5):
        service_idx = 5 + (i-2)*2
        db_call_idx = 6 + (i-2)*2
        activities[service_idx] = Activity(model, f'ServiceActivity{i}', Exp.fit_mean(0.5 + i * 0.2)).on(tasks[i]).bound_to(entries[i])
        activities[db_call_idx] = Activity(model, f'DBCall{i}', Immediate()).on(tasks[i]).synch_call(entries[5], 1).replies_to(entries[i])
        tasks[i].add_precedence(ActivityPrecedence.Serial(activities[service_idx], activities[db_call_idx]))
    
    # Database activity
    activities[11] = Activity(model, 'DatabaseActivity', Exp.fit_mean(2.0)).on(tasks[5]).bound_to(entries[5]).replies_to(entries[5])
    
    # Add precedences for client task
    tasks[1].add_precedence(ActivityPrecedence.Serial(activities[1], activities[2]))
    tasks[1].add_precedence(ActivityPrecedence.Serial(activities[2], activities[3]))
    tasks[1].add_precedence(ActivityPrecedence.Serial(activities[3], activities[4]))
    
    print('Large layered model created (ofbizExample substitute)')

In [3]:
# MATLAB: options = LQNS.defaultOptions;
# MATLAB: options.keep = true; % uncomment to keep the intermediate XML files generates while translating the model to LQNS
options = LQNS.default_options()
options.keep = True  # uncomment to keep the intermediate XML files generates while translating the model to LQNS

# Solve with LN without initialization (MATLAB comment, but this is actually LQNS)
# MATLAB: solver{1} = LQNS(model);
# MATLAB: AvgTable{1} = solver{1}.get_avg_table;
# MATLAB: AvgTable{1}
solver = {}
AvgTable = {}

if LQNS.isAvailable():
    solver[1] = LQNS(model)
    AvgTable[1] = solver[1].avg_table()
    print('AvgTable[1]:')
    print(AvgTable[1])
else:
    print('LQNS solver not available - skipping solver[1]')

# Solve with LN without initialization
# MATLAB: solver{2} = LN(model, @(x) NC(x,'verbose',false));
# MATLAB: Tnoinit = tic;
# MATLAB: AvgTable{2} = solver{2}.get_avg_table;
# MATLAB: AvgTable{2}
# MATLAB: Tnoinit = toc(Tnoinit)
import time
nc_options = NC.default_options()
nc_options.verbose = False  # 'verbose',false in MATLAB
solver[2] = LN(model, lambda x: NC(x, nc_options))
Tnoinit_start = time.time()
AvgTable[2] = solver[2].avg_table()
Tnoinit = time.time() - Tnoinit_start
print(f'Tnoinit = {Tnoinit:.6f}')

                Node   NodeType    QLen        Util    RespT  ResidT  ArvR  \
0                 P1  Processor     NaN  1.6689e-02      NaN     0.0   0.0   
1                 P2  Processor     NaN  1.5020e-01      NaN     0.0   0.0   
2                 P3  Processor     NaN  1.8358e-01      NaN     0.0   0.0   
3                 P4  Processor     NaN  2.1695e-01      NaN     0.0   0.0   
4                 P5  Processor     NaN  1.0013e+00      NaN     0.0   0.0   
5         ClientTask    RefTask  9.8331  1.6689e-02      NaN     0.0   0.0   
6       ServiceTask2       Task  0.7960  1.5020e-01      NaN     0.0   0.0   
7       ServiceTask3       Task  0.8284  1.8358e-01      NaN     0.0   0.0   
8       ServiceTask4       Task  0.8614  2.1695e-01      NaN     0.0   0.0   
9       DatabaseTask       Task  0.9612  1.0013e+00      NaN     0.0   0.0   
10       ClientEntry      Entry  9.8331  1.6689e-02  58.9207     0.0   0.0   
11     ServiceEntry2      Entry  0.7960  1.5020e-01   4.7696    